# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [45]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [46]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [47]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [48]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [49]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [50]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [51]:
def jacobi(A, b, n_iter=100):
    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    D = np.diag(diag_1d)
    invD = np.diag(1. / diag_1d)
    BB = invD @ B 
    print(np.linalg.norm(BB))
    c = invD @ b
    x = np.ones_like(b)
    
    for i in range(n_iter):
        x = BB @ x + c
        
    return x


In [52]:
#checking the result
A = rndm.uniform(size=(n, n)) + np.diagflat([25]*n)
b = rndm.uniform(size=n)

x = jacobi(A, b)
x_ = np.linalg.solve(A, b)
np.testing.assert_allclose(x, x_)

0.22434016905655696


In [53]:
#checking the result for smaller diagonal elements
A = rndm.uniform(size=(n, n)) + np.diagflat([1]*n)
b = rndm.uniform(size=n)

x = jacobi(A, b)
x_ = np.linalg.solve(A, b)
np.testing.assert_allclose(x, x_)

3.4341680304660542


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatch: 100%
Max absolute difference: 3.03657812e+44
Max relative difference: 1.1885024e+46
 x: array([2.000284e+44, 2.077820e+44, 1.625835e+44, 3.036578e+44,
       2.057696e+44, 1.369375e+44, 1.342124e+44, 2.001294e+44,
       1.490639e+44, 1.472647e+44])
 y: array([-0.159365,  0.017483,  0.367765,  0.756746,  0.199009, -0.196033,
        0.552569, -0.654353, -0.223896,  0.277508])

In [54]:
# We see that if the matrix is no longer diagonally dominant, the value of the norm of B is greater than 1 and Jacobi iteration does not converge.

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [55]:
def seidel(A, b, n_iter=100):
    U = np.triu(A, k=1)
    L = np.tril(A)
    invL = np.linalg.inv(L)
    BB = -invL @ U
    print(np.linalg.norm(BB))
    x = np.ones_like(b)
    
    for i in range(n_iter):
        x = invL @ (b - U @ x)
        
    return x

In [56]:
#checking the result
A = rndm.uniform(size=(n, n)) + np.diagflat([25]*n)
b = rndm.uniform(size=n)

x = seidel(A, b)
x_ = np.linalg.solve(A, b)
np.testing.assert_allclose(x, x_)

0.16012535332251066


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [57]:
def residual(A, b, n_iter=100):
    x = np.ones_like(b)
    tau = 0
    
    for i in range(n_iter):
        res = A @ x - b
        d = A @ res
        tau = np.inner(d, res) / np.inner(d, d)   
        x = x - tau * res
        
    return x

In [58]:
#checking the result
A = rndm.uniform(size=(n, n)) + np.diagflat([25]*n)
b = rndm.uniform(size=n)

x = residual(A, b)
x_ = np.linalg.solve(A, b)
np.testing.assert_allclose(x, x_)